In [293]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [348]:
import pandas as pd
import glob, os, time, collections
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv
from nltk import agreement
from statsmodels.stats import inter_rater

In [349]:
path = r'results/ratings_overview/' 
allFiles = glob.glob(path + "/*.csv")
list_ = []
for f in allFiles:
    df = pd.read_csv(f,index_col=None, header=0)
    list_.append(df)
feedback = pd.concat(list_)

In [350]:
path = r'results/dataset_entities/' 
allFiles = glob.glob(path + "/*.txt")
dataset = {}
for f in allFiles:
    name = 'datset'
    entities = pd.read_csv(f, index_col=None, header=None)
    dataset[name] = set(list(entities[0]))
    
path = r'results/method_entities/' 
allFiles = glob.glob(path + "/*.txt")
method = {}
for f in allFiles:
    name = 'method'
    entities = pd.read_csv(f, index_col=None, header=None)
    method[name] = set(list(entities[0]))

In [351]:
method_annotated = feedback.loc[(feedback['facet'] == "method")]
dataset_annotated = feedback.loc[(feedback['facet'] == "dataset")]

# dataset_annotated

- What % of entities did get user feedback? 
- For those entities which have feedback from 2 or more people, what is the inter anotator agreement?
- What % of dataset / method entities where considered correct / incorrect by the majority? (note: there is not necessarily always a majority)

In [352]:
# i = 0

# for paper in dataset.keys():
#     print(paper)
#     print(len(dataset_annotated.loc[(dataset_annotated['paper_id'] == "'" + paper + "'")].groupby(['entity_text'])['paper_id'].count()))
#     print(len(dataset[paper]))
#     print(len(dataset_annotated.loc[(dataset_annotated['paper_id'] == "'" + paper + "'")].groupby(['entity_text'])['paper_id'].count()) / len(dataset[paper]))
#     i = len(dataset[paper]) + i

# print('')
# print(len(dataset_annotated.entity_text.unique()) / i)

In [353]:
57/3

19.0

In [354]:
dataset_relevant = dataset_annotated.loc[(dataset_annotated['relevant'] == "relevant")].groupby(['entity_text']).count()
len(dataset_relevant)

138

In [355]:
16/57

0.2807017543859649

In [356]:
# i = 0
# for paper in method.keys():
#     print(paper)
#     print(len(method_annotated.loc[(method_annotated['paper_id'] == "'" + paper + "'")].groupby(['entity_text'])['paper_id'].count()))
#     print(len(method[paper]))
#     print(len(method_annotated.loc[(method_annotated['paper_id'] == "'" + paper + "'")].groupby(['entity_text'])['paper_id'].count()) / len(method[paper]))
#     i = len(method[paper]) + i

# print('')
# print(len(method_annotated.entity_text.unique()) / i)

In [357]:
method_relevant = method_annotated.loc[(method_annotated['relevant'] == "relevant")].groupby(['entity_text']).count() 
len(method_relevant)

262

In [358]:
27/74

# print(dataset_annotated)

0.36486486486486486

<H2> Annotator Agreement </H2>

<H3> Dataset Entities </H3>

In [359]:
cleanup_nums = {"relevant": {"relevant": 1, "irrelevant": 0}}
dataset_annotated.replace(cleanup_nums, inplace=True)



/Applications/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


In [360]:
dataset_agreement = pd.DataFrame(index = dataset_annotated.entity_text.unique())

pairs_count = 0
kappas = []

users = dataset_annotated.user_id.unique()
for usr1 in dataset_annotated.user_id.unique():
    ann1 = dataset_annotated.loc[dataset_annotated['user_id'] == usr1][['entity_text', 'relevant']]
    ann1 = ann1.rename(columns={'relevant' : usr1})
    
    for usr2 in dataset_annotated.user_id.unique():
        if(usr1 == usr2): continue
        dataset_agreement = pd.DataFrame(index = dataset_annotated.entity_text.unique())
        dataset_agreement = dataset_agreement.merge(ann1, left_index=True, right_on='entity_text', how='left').set_index('entity_text')

        annotation = dataset_annotated.loc[dataset_annotated['user_id'] == usr2][['entity_text', 'relevant']]
        annotation = annotation.rename(columns={'relevant' : usr2})
        dataset_agreement = dataset_agreement.merge(annotation, left_index=True, right_on='entity_text', how='left').set_index('entity_text')
    
        df = dataset_agreement[~dataset_agreement.index.duplicated(keep='first')]
        df['null_count'] = df.isnull().sum(axis=1)
        df = df.sort_values('null_count', ascending=True)
        dataset_agreement = df.drop('null_count', axis=1).dropna()
        if(dataset_agreement.shape[0] > 20):
            pairs_count+=1
            taskdata = ([[0,str(i),str(dataset_agreement[usr1][i])] for i in range(0, len(dataset_agreement))] + [[1,str(i),str(dataset_agreement[usr2][i])] for i in range(0, len(dataset_agreement))])
#             print(taskdata)
            ratingtask = agreement.AnnotationTask(data=taskdata)
            try: 
                kappa_val = ratingtask.kappa()
            
#                 print("kappa " +str(kappa_val))
                if not (kappa_val == 0.0 or kappa_val == 1.0): kappas.append(kappa_val)
            except:
                continue
                
print(sum(kappas)/len(kappas))
print(len(kappas))
                    






/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.5166099089742238
76


In [361]:
cleanup_nums = {"relevant":     {"relevant": 1, "irrelevant": 0}}
method_annotated.replace(cleanup_nums, inplace=True)

/Applications/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


In [364]:
method_agreement = pd.DataFrame(index = method_annotated.entity_text.unique())

pairs_count = 0
kappas = []

users = method_annotated.user_id.unique()
for usr1 in method_annotated.user_id.unique():
    ann1 = method_annotated.loc[method_annotated['user_id'] == usr1][['entity_text', 'relevant']]
    ann1 = ann1.rename(columns={'relevant' : usr1})
    
    for usr2 in method_annotated.user_id.unique():
        if(usr1 == usr2): continue
        method_agreement = pd.DataFrame(index = method_annotated.entity_text.unique())
        method_agreement = method_agreement.merge(ann1, left_index=True, right_on='entity_text', how='left').set_index('entity_text')

        annotation = method_annotated.loc[method_annotated['user_id'] == usr2][['entity_text', 'relevant']]
        annotation = annotation.rename(columns={'relevant' : usr2})
        method_agreement = method_agreement.merge(annotation, left_index=True, right_on='entity_text', how='left').set_index('entity_text')
    
        df = method_agreement[~method_agreement.index.duplicated(keep='first')]
        df['null_count'] = df.isnull().sum(axis=1)
        df = df.sort_values('null_count', ascending=True)
        method_agreement = df.drop('null_count', axis=1).dropna()
        if(method_agreement.shape[0] > 20):
            pairs_count+=1
            taskdata = ([[0,str(i),str(method_agreement[usr1][i])] for i in range(0, len(method_agreement))] + [[1,str(i),str(method_agreement[usr2][i])] for i in range(0, len(method_agreement))])
#             print(taskdata)
            ratingtask = agreement.AnnotationTask(data=taskdata)
            try: 
                kappa_val = ratingtask.kappa()
            
#                 print("kappa " +str(kappa_val))
                if not (kappa_val == 0.0 or kappa_val == 1.0): kappas.append(kappa_val)
            except:
                continue
                
print(sum(kappas)/len(kappas))
print(len(kappas))

/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.6384723079974136
68


In [188]:
# dataset_agreement = pd.DataFrame(index = dataset_annotated.entity_text.unique())

# for user in dataset_annotated.user_id.unique():
#    annotation = dataset_annotated.loc[dataset_annotated['user_id'] == user][['entity_text', 'relevant']]
#    annotation = annotation.rename(columns={'relevant' : user})
# #    if(user == "diuSsjdsEoWWDTejfcpU6vaAbjt1"): print(annotation)
#    dataset_agreement = dataset_agreement.merge(annotation, left_index=True, right_on='entity_text', how='left').set_index('entity_text')
    
#    dataset_agreement_t = dataset_agreement.transpose()

In [189]:
# dataset_agreement = dataset_agreement.as_matrix()
# dataset_agreement = dataset_agreement.values
# dataset_agreement_t = dataset_agreement_t.values

# dataset_agreement

In [190]:
# inter_rater.fleiss_kappa(dataset_agreement_t)

In [278]:
df = dataset_agreement[~dataset_agreement.index.duplicated(keep='first')]
df['null_count'] = df.isnull().sum(axis=1)
df = df.sort_values('null_count', ascending=True)
df_temp = df
dataset_agreement = df.drop('null_count', axis=1)
selected_users = dataset_agreement.ix[:, :7].dropna(thresh=6)
selected_users

,diuSsjdsEoWWDTejfcpU6vaAbjt1,YdJDigrdTZUOiUKMsMv1ciWxHtc2,Nq119A7Qa4hPWNSThUX3S8Y8k313,qV1wemRqtjWRBxqOnqDpbEDE7dQ2,XnqHi69zi9gmO4e7xFe6aK9PEiE3,G8hHT8mWZiXmaJTPIyLj6KdjzVF2,lqmIlg05OpMYR8f6VR9UPN29nWg2
entity_text,,,,,,,
whether,0.0,0.0,0.0,0.0,0.0,0.0,0.0
slashdot,1.0,1.0,1.0,1.0,1.0,1.0,1.0
wikipedia,1.0,1.0,1.0,1.0,1.0,0.0,1.0
social network,0.0,1.0,1.0,0.0,1.0,0.0,0.0
related work,0.0,0.0,0.0,0.0,0.0,0.0,0.0
social networks,0.0,0.0,1.0,0.0,1.0,0.0,1.0
although,0.0,0.0,0.0,0.0,0.0,0.0,0.0
epinions,1.0,1.0,1.0,1.0,1.0,NaN,1.0
without,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [276]:
users = ["diuSsjdsEoWWDTejfcpU6vaAbjt1", "YdJDigrdTZUOiUKMsMv1ciWxHtc2", "Nq119A7Qa4hPWNSThUX3S8Y8k313", "qV1wemRqtjWRBxqOnqDpbEDE7dQ2", "XnqHi69zi9gmO4e7xFe6aK9PEiE3", "lqmIlg05OpMYR8f6VR9UPN29nWg2"]
print('users =', users)

dataset_agreement1 = selected_users[users]
raters = [list(dataset_agreement1["diuSsjdsEoWWDTejfcpU6vaAbjt1"])
        ,list(dataset_agreement1["YdJDigrdTZUOiUKMsMv1ciWxHtc2"])
        ,list(dataset_agreement1["Nq119A7Qa4hPWNSThUX3S8Y8k313"])
        ,list(dataset_agreement1["qV1wemRqtjWRBxqOnqDpbEDE7dQ2"])
        ,list(dataset_agreement1["XnqHi69zi9gmO4e7xFe6aK9PEiE3"])
#         ,list(dataset_agreement1["G8hHT8mWZiXmaJTPIyLj6KdjzVF2"])
        ,list(dataset_agreement1["lqmIlg05OpMYR8f6VR9UPN29nWg2"])
         ]

# taskdata = ([[0,str(i),str(rater1[i])] for i in xrange(0, len(dataset_agreement1))] 
#             + [[1,str(i),str(rater2[i])] for i in range(0, len(dataset_agreement1))]
#            + [[1,str(i),str(rater3[i])] for i in range(0, len(dataset_agreement1))]
#            + [[1,str(i),str(rater4[i])] for i in range(0, len(dataset_agreement1))]
#            + [[1,str(i),str(rater5[i])] for i in range(0, len(dataset_agreement1))]
#            + [[1,str(i),str(rater6[i])] for i in range(0, len(dataset_agreement1))]
#            + [[1,str(i),str(rater7[i])] for i in range(0, len(dataset_agreement1))]
#            )
# print(taskdata)

kappas = []
for usr1 in range(0, len(users)):
    for usr2 in range(0, len(users)):
        if(usr1 == usr2 or usr1 < usr2 or (usr1 == 3 and usr2 == 0)): continue
        taskdata = ([[usr1,str(i),str(raters[usr1][i])] for i in range(0, len(dataset_agreement1))] + [[usr2,str(i),str(raters[usr2][i])] for i in range(0, len(dataset_agreement1))])
        ratingtask = agreement.AnnotationTask(data=taskdata)
        kappa_val = ratingtask.kappa()
        print("kappa " +str(kappa_val))
        kappas.append(kappa_val)
        

#print('entities annotated =', dataset_agreement[["'user_2'", "'user_6'", "'user_8'", "'user_9'"]].iloc[:16].index)

# ratingtask = agreement.AnnotationTask(data=taskdata)
# print("kappa " +str(ratingtask.kappa()))
# print("fleiss " + str(ratingtask.multi_kappa()))
#print("alpha " +str(ratingtask.alpha()))
#print("scotts " + str(ratingtask.pi()))
print('')

print(len(kappas))
print(sum(kappas)/len(kappas))


users = ['diuSsjdsEoWWDTejfcpU6vaAbjt1', 'YdJDigrdTZUOiUKMsMv1ciWxHtc2', 'Nq119A7Qa4hPWNSThUX3S8Y8k313', 'qV1wemRqtjWRBxqOnqDpbEDE7dQ2', 'XnqHi69zi9gmO4e7xFe6aK9PEiE3', 'lqmIlg05OpMYR8f6VR9UPN29nWg2']
kappa 0.5
kappa 0.5882352941176469
kappa 0.6666666666666666
kappa 0.5
kappa 0.5882352941176469
kappa 0.3913043478260871
kappa 0.4545454545454546
kappa 0.7199999999999999
kappa 0.3913043478260871
kappa 0.6956521739130435
kappa 0.5384615384615385
kappa 0.8771929824561402
kappa 0.6956521739130435
kappa 0.6164383561643837

14
0.5874063307148384


In [ ]:
(0.2 + 0.8518518518518519 + 0.8461538461538461 + 0.12195121951219512 + 0.05263157894736842 + 0.8608695652173913) / 6

In [115]:
dataset_agreement1 = dataset_agreement[["'user_1'", "'user_4'", "'user_10'"]].iloc[22:35]

rater1 = list(dataset_agreement1["'user_1'"])
rater4 = list(dataset_agreement1["'user_4'"])
rater10 = list(dataset_agreement1["'user_10'"])

taskdata = [[0,str(i),str(rater1[i])] for i in range(0,len(rater1))] + [[1,str(i),str(rater4[i])] for i in range(0,len(rater4))] + [[2,str(i),str(rater10[i])] for i in range(0,len(rater10))]

print('users =', "'user_1'", "'user_4'", "'user_10'")
#print('entities annotated =', dataset_agreement[["'user_1'", "'user_4'", "'user_10'"]].iloc[22:35].index)

ratingtask = agreement.AnnotationTask(data=taskdata)
print("fleiss " + str(ratingtask.multi_kappa()))

# users = ["'user_1'", "'user_4'", "'user_10'"]

# for user in users:
#     for user2 in users:
#         if user != user2:
#             taskdata = ([[0, str(i), str(list(dataset_agreement1[user])[i])] for i in range(0, len(dataset_agreement1))] 
#                         + [[1,str(i),str(list(dataset_agreement1[user2])[i])] for i in range(0, len(dataset_agreement1))])
#             print('users =', user, user2)
#             ratingtask = agreement.AnnotationTask(data=taskdata)
#             print("kappa " +str(ratingtask.kappa()))
#             print('')

KeyError: '["\'user_1\'" "\'user_4\'" "\'user_10\'"] not in index'

In [116]:
(0.09900990099009897 + 0.5301204819277109 + 0.1333333333333332) / 3

0.25415457208371434

In [663]:
dataset_agreement1 = dataset_agreement[["'user_3'", "'user_5'", "'user_7'"]].iloc[16:22]

rater3 = list(dataset_agreement1["'user_3'"])
rater5 = list(dataset_agreement1["'user_5'"])
rater7 = list(dataset_agreement1["'user_7'"])

taskdata = [[0,str(i),str(rater3[i])] for i in range(0,len(rater3))] + [[1,str(i),str(rater5[i])] for i in range(0,len(rater5))] + [[2,str(i),str(rater7[i])] for i in range(0,len(rater7))]

print('users =', "'user_3'", "'user_5'", "'user_7'")
#print('entities annotated =', dataset_agreement[["'user_3'", "'user_5'", "'user_7'"]].iloc[16:22].index)

ratingtask = agreement.AnnotationTask(data=taskdata)
#print("kappa " +str(ratingtask.kappa()))
print("fleiss " + str(ratingtask.multi_kappa()))
#print("alpha " +str(ratingtask.alpha()))
#print("scotts " + str(ratingtask.pi()))
print('')

users = ["'user_3'", "'user_5'", "'user_7'"]

for user in users:
    for user2 in users:
        if user != user2:
            taskdata = ([[0, str(i), str(list(dataset_agreement1[user])[i])] for i in range(0, len(dataset_agreement1))] 
                        + [[1,str(i),str(list(dataset_agreement1[user2])[i])] for i in range(0, len(dataset_agreement1))])
            print('users =', user, user2)
            ratingtask = agreement.AnnotationTask(data=taskdata)
            print("kappa " +str(ratingtask.kappa()))
            print('')

users = 'user_3' 'user_5' 'user_7'
fleiss 0.35714285714285704

users = 'user_3' 'user_5'
kappa 0.6666666666666667

users = 'user_3' 'user_7'
kappa 0.0

users = 'user_5' 'user_3'
kappa 0.6666666666666667

users = 'user_5' 'user_7'
kappa 0.39999999999999997

users = 'user_7' 'user_3'
kappa 0.0

users = 'user_7' 'user_5'
kappa 0.39999999999999997



In [671]:
(0.6666666666666667 + 0.39999999999999997) / 3

0.35555555555555557

In [672]:
(0.48890967694710885 + 0.25415457208371434 + 0.35555555555555557) / 3

0.366206601528793

<H3> Method Entities </H3>

In [502]:
cleanup_nums = {"relevant":     {"relevant": 1, "irrelevant": 0}}
method_annotated.replace(cleanup_nums, inplace=True)

C:\Users\mvall\Anaconda3\lib\site-packages\pandas\core\generic.py:3744: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


In [607]:
method_agreement = pd.DataFrame(index = method_annotated.entity_text.unique())

for user in method_annotated.user_id.unique():
    annotation = method_annotated.loc[method_annotated['user_id'] == user][['entity_text', 'relevant']]
    annotation = annotation.rename(columns={'relevant' : user})
    method_agreement = method_agreement.merge(annotation, left_index=True, right_on='entity_text', how='left').set_index('entity_text')
    method_agreement_t = method_agreement.transpose()

In [664]:
method_agreement1 = method_agreement[["'user_2'", "'user_6'", "'user_8'", "'user_9'"]].iloc[0:16]

rater2 = list(method_agreement["'user_2'"])
rater6 = list(method_agreement["'user_6'"])
rater8 = list(method_agreement["'user_8'"])
rater9 = list(method_agreement["'user_9'"])

taskdata = ([[0,str(i),str(rater2[i])] for i in range(0,len(method_agreement1))] + [[1,str(i),str(rater6[i])] for i in range(0,len(method_agreement1))] +
        [[2,str(i),str(rater9[i])] for i in range(0,len(method_agreement1))] + [[3,str(i),str(rater8[i])] for i in range(0,len(method_agreement1))])

ratingtask = agreement.AnnotationTask(data=taskdata)
#print("kappa " +str(ratingtask.kappa()))
print("fleiss " + str(ratingtask.multi_kappa()))
#print("alpha " +str(ratingtask.alpha()))
#print("scotts " + str(ratingtask.pi()))

print('')

users = ["'user_2'", "'user_6'", "'user_8'", "'user_9'"]

for user in users:
    for user2 in users:
        if user != user2:
            taskdata = ([[0, str(i), str(list(method_agreement1[user])[i])] for i in range(0, len(method_agreement1))] 
                        + [[1,str(i),str(list(method_agreement1[user2])[i])] for i in range(0, len(method_agreement1))])
            print('users =', user, user2)
            ratingtask = agreement.AnnotationTask(data=taskdata)
            print("kappa " +str(ratingtask.kappa()))
            print('')

fleiss 0.2461719670200236

users = 'user_2' 'user_6'
kappa 0.14285714285714285

users = 'user_2' 'user_8'
kappa 0.33884297520661155

users = 'user_2' 'user_9'
kappa 0.022900763358778626

users = 'user_6' 'user_2'
kappa 0.14285714285714285

users = 'user_6' 'user_8'
kappa 0.13513513513513514

users = 'user_6' 'user_9'
kappa 0.2

users = 'user_8' 'user_2'
kappa 0.33884297520661155

users = 'user_8' 'user_6'
kappa 0.13513513513513514

users = 'user_8' 'user_9'
kappa 0.6595744680851063

users = 'user_9' 'user_2'
kappa 0.022900763358778626

users = 'user_9' 'user_6'
kappa 0.2

users = 'user_9' 'user_8'
kappa 0.6595744680851063



In [673]:
(0.14285714285714285 + 0.33884297520661155 + 0.022900763358778626 + 0.13513513513513514 + 0.2 + 0.6595744680851063) / 6

0.24988508077379576

In [667]:
method_agreement1 = method_agreement[["'user_3'", "'user_5'", "'user_7'"]].iloc[17:32]

rater3 = list(method_agreement1["'user_3'"])
rater5 = list(method_agreement1["'user_5'"])
rater7 = list(method_agreement1["'user_7'"])

taskdata = ([[0,str(i),str(rater3[i])] for i in range(0,len(rater3))] + [[1,str(i),str(rater5[i])] for i in range(0,len(rater5))] +
        [[2,str(i),str(rater7[i])] for i in range(0,len(rater7))])

ratingtask = agreement.AnnotationTask(data=taskdata)
#print("kappa " +str(ratingtask.kappa()))
print("fleiss " + str(ratingtask.multi_kappa()))
#print("alpha " +str(ratingtask.alpha()))
#print("scotts " + str(ratingtask.pi()))

print('')

users = ["'user_3'", "'user_5'", "'user_7'"]

for user in users:
    for user2 in users:
        if user != user2:
            taskdata = ([[0, str(i), str(list(method_agreement1[user])[i])] for i in range(0, len(method_agreement1))] 
                        + [[1,str(i),str(list(method_agreement1[user2])[i])] for i in range(0, len(method_agreement1))])
            print('users =', user, user2)
            ratingtask = agreement.AnnotationTask(data=taskdata)
            print("kappa " +str(ratingtask.kappa()))
            print('')

fleiss 0.38596491228070157

users = 'user_3' 'user_5'
kappa 0.3589743589743589

users = 'user_3' 'user_7'
kappa 0.4736842105263157

users = 'user_5' 'user_3'
kappa 0.3589743589743589

users = 'user_5' 'user_7'
kappa 0.3243243243243242

users = 'user_7' 'user_3'
kappa 0.4736842105263157

users = 'user_7' 'user_5'
kappa 0.3243243243243242



In [674]:
(0.3589743589743589 + 0.4736842105263157 + 0.3243243243243242) / 3

0.38566096460833293

In [668]:
method_agreement1 = method_agreement[["'user_1'", "'user_4'", "'user_10'"]].iloc[32:55]

rater1 = list(method_agreement1["'user_1'"])
rater4 = list(method_agreement1["'user_4'"])
rater10 = list(method_agreement1["'user_10'"])

taskdata = ([[0,str(i),str(rater1[i])] for i in range(0,len(rater1))] + [[1,str(i),str(rater4[i])] for i in range(0,len(rater4))] +
        [[2,str(i),str(rater10[i])] for i in range(0,len(rater10))])

ratingtask = agreement.AnnotationTask(data=taskdata)
#print("kappa " +str(ratingtask.kappa()))
print("fleiss " + str(ratingtask.multi_kappa()))
#print("alpha " +str(ratingtask.alpha()))
#print("scotts " + str(ratingtask.pi()))

print('')

users = ["'user_1'", "'user_4'", "'user_10'"]

for user in users:
    for user2 in users:
        if user != user2:
            taskdata = ([[0, str(i), str(list(method_agreement1[user])[i])] for i in range(0, len(method_agreement1))] 
                        + [[1,str(i),str(list(method_agreement1[user2])[i])] for i in range(0, len(method_agreement1))])
            print('users =', user, user2)
            ratingtask = agreement.AnnotationTask(data=taskdata)
            print("kappa " +str(ratingtask.kappa()))
            print('')

fleiss 0.3251173708920189

users = 'user_1' 'user_4'
kappa 0.554263565891473

users = 'user_1' 'user_10'
kappa 0.1732026143790849

users = 'user_4' 'user_1'
kappa 0.554263565891473

users = 'user_4' 'user_10'
kappa 0.2812500000000001

users = 'user_10' 'user_1'
kappa 0.1732026143790849

users = 'user_10' 'user_4'
kappa 0.2812500000000001



In [675]:
(0.554263565891473 + 0.1732026143790849 + 0.2812500000000001) / 3

0.3362387267568527

In [676]:
(0.24988508077379576 + 0.38566096460833293 + 0.3362387267568527 ) / 3

0.32392825737966047

<H3> All Entities </H3>

In [678]:
paper1_agreement = pd.concat([dataset_agreement[["'user_2'", "'user_6'", "'user_8'", "'user_9'"]].iloc[:16], method_agreement[["'user_2'", "'user_6'", "'user_8'", "'user_9'"]].iloc[:16]])

rater2 = list(paper1_agreement["'user_2'"])
rater6 = list(paper1_agreement["'user_6'"])
rater8 = list(paper1_agreement["'user_8'"])
rater9 = list(paper1_agreement["'user_9'"])

taskdata = ([[0,str(i),str(rater2[i])] for i in range(0,len(rater2))] + [[1,str(i),str(rater6[i])] for i in range(0,len(rater2))] +
        [[2,str(i),str(rater9[i])] for i in range(0, len(rater2))] + [[3,str(i),str(rater8[i])] for i in range(0, len(rater2))])

ratingtask = agreement.AnnotationTask(data=taskdata)
print("fleiss " + str(ratingtask.multi_kappa()))

fleiss 0.4221668742216687


In [684]:
paper1_agreement = pd.concat([dataset_agreement[["'user_1'", "'user_4'", "'user_10'"]].iloc[22:35], method_agreement[["'user_1'", "'user_4'", "'user_10'"]].iloc[32:55]])

rater1 = list(paper1_agreement["'user_1'"])
rater4 = list(paper1_agreement["'user_4'"])
rater10 = list(paper1_agreement["'user_10'"])

taskdata = ([[0,str(i),str(rater1[i])] for i in range(0,len(rater1))] + [[1,str(i),str(rater4[i])] for i in range(0,len(rater4))] +
        [[2,str(i),str(rater10[i])] for i in range(0, len(rater10))] )

ratingtask = agreement.AnnotationTask(data=taskdata)
print("fleiss " + str(ratingtask.multi_kappa()))

fleiss 0.3004739336492891


In [686]:
paper1_agreement = pd.concat([method_agreement[["'user_3'", "'user_5'", "'user_7'"]].iloc[17:32], dataset_agreement[["'user_3'", "'user_5'", "'user_7'"]].iloc[16:22]])

rater1 = list(paper1_agreement["'user_3'"])
rater4 = list(paper1_agreement["'user_5'"])
rater10 = list(paper1_agreement["'user_7'"])

taskdata = ([[0,str(i),str(rater1[i])] for i in range(0,len(rater1))] + [[1,str(i),str(rater4[i])] for i in range(0,len(rater4))] +
        [[2,str(i),str(rater10[i])] for i in range(0, len(rater10))] )

ratingtask = agreement.AnnotationTask(data=taskdata)
print("fleiss " + str(ratingtask.multi_kappa()))

fleiss 0.36936936936936915


In [687]:
 (0.4221668742216687 +0.3004739336492891 +0.36936936936936915) / 3

0.3640033924134423